In [1]:
# Notebook til statiske figurer af covid fremskrivning
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")

import os
import math

from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]


saveFigures is set to: True
Done loading packages


# Load real data

In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() + '/' + ssidatapath
dfAdm = pd.read_excel('Admitted\Admitted.xlsx')
dfAdm = dfAdm.transpose()
dfAdm.columns = dfAdm.iloc[0]
dfAdm = dfAdm.drop(['Aldersgruppe']) 
curDates =  pd.to_datetime(dfAdm.index,format='%d_%m_%Y')
diffDates = curDates[1:]
dfCase = pd.DataFrame(columns=dfAdm.columns)

for k in range(0,len(curDates)):
    d = curDates[k]
    curIndex = dfAdm.index[k]

    thisDateStr = d.strftime('%Y-%m-%d')
    curDir = rootdir + '/SSI_data_' + thisDateStr 
    curFilePath = curDir + '/Cases_by_age.csv'
    casedatadf = pd.read_csv(curFilePath,delimiter=';',dtype=str)

    casedatadf['Antal_bekræftede_COVID-19'] = pd.to_numeric(casedatadf['Antal_bekræftede_COVID-19'].astype(str).apply(lambda x: x.replace('.','')))

    dfCase.loc[curIndex] = casedatadf['Antal_bekræftede_COVID-19'].values

In [3]:
# Same things, but with cases
dfCaseDiff = dfCase.diff().iloc[1:]
dfCaseDiff['Dato'] = curDates[1:]
# Bad hack to ignore 10-15 year olds!
dfCaseDiff['10-19'] = np.multiply(dfCaseDiff['10-19'],0.4)
dfCaseDiffSum = dfCaseDiff.iloc[:,1:-2].cumsum(axis=1)
dfCaseDiffSum['Dato'] = curDates[1:]

# dfCaseDiff = dfCaseDiff.iloc[:-1]
# dfCaseDiffSum = dfCaseDiffSum.iloc[:-1]

dfCaseDiffSum['I alt'] = dfCaseDiffSum['90+']


dfAdmDiff = dfAdm.diff().iloc[1:]
dfAdmDiff['Dato'] = curDates[1:]
dfAdmDiffSum = dfAdmDiff.iloc[:,1:-2].cumsum(axis=1)
dfAdmDiffSum['Dato'] = curDates[1:]

# dfAdmDiff = dfAdmDiff.iloc[:-1]
# dfAdmDiffSum = dfAdmDiffSum.iloc[:-1]
dfAdmDiffSum['I alt'] = dfAdmDiffSum['90+']


# Load model data

In [4]:
datafilename = 'model_results_spread_prediction_var_vacc.csv'
df = pd.read_csv(datafilename)
df.End =  pd.to_datetime(df.End,format='%Y-%m-%d')
df.Start =  pd.to_datetime(df.Start,format='%Y-%m-%d')
df['RT2'] = np.round(np.multiply(df.RT,10)).astype(int)
df['Hosp'] = np.multiply(df.Count,df.CaseHospRate)

datafilenameD = 'model_results_spread_prediction_var_vacc_delayed.csv'
dfD = pd.read_csv(datafilenameD)
dfD.End =  pd.to_datetime(dfD.End,format='%Y-%m-%d')
dfD.Start =  pd.to_datetime(dfD.Start,format='%Y-%m-%d')
dfD['RT2'] = np.round(np.multiply(dfD.RT,10)).astype(int)
dfD['Hosp'] = np.multiply(dfD.Count,dfD.CaseHospRate)

In [5]:
vaccToUse = 0.75
df = df[df.vacc1 == vaccToUse]
dfD = dfD[dfD.vacc1 == vaccToUse]

In [6]:
RT2_to_use = 13
dfRT = df[df.RT2 == RT2_to_use]
dfRTD = dfD[dfD.RT2 == RT2_to_use]

In [7]:
# # Calculate totals
# allDays = df.End.unique()

# totCount = []
# totHosp = []

# for curDay in allDays:
#     totCount.append(df[df.End == curDay].Count.sum())
#     totHosp.append(df[df.End == curDay].Hosp.sum())
    
# allDaysD = dfD.End.unique()

# totCountD = []
# totHospD = []

# for curDay in allDaysD:
#     totCountD.append(dfD[dfD.End == curDay].Count.sum())
#     totHospD.append(dfD[dfD.End == curDay].Hosp.sum())


In [8]:
# dfCaseDiff.tail()

In [9]:
# Plot

 
allRTsToUse = np.arange(9,18,1)

# RT2_to_use = 13
for RT2_to_use in allRTsToUse:
    dfRT = df[df.RT2 == RT2_to_use]
    dfRTD = dfD[dfD.RT2 == RT2_to_use]

    fig,(ax1,ax2) = plt.subplots(2,1,sharex=True,figsize=(12,12))

    allAges = df.Age.unique()
    cmap = plt.cm.get_cmap('terrain',len(allAges)+1)




    # # Data
    # curCountData = np.zeros(dfCaseDiff['10-19'].values.shape)
    # prevCountData = np.zeros(dfCaseDiff['10-19'].values.shape)

    # for k in range(0,len(allAges)):
    #     curAge = allAges[k]
    #     curColor = cmap(k)

    #     # curCountData = np.add(dfCaseDiff[curAge].values,curCountData)
    #     # ax1.plot(diffDates,curCountData,'*-',color=curColor,linewidth=0.5)
    #     # prevCountData = curCountData

        
    #     # curCountData = np.add(dfCaseDiff[curAge].values,curCountData)
    #     # ax2.plot(diffDates,curCountData,'*-',color=curColor,linewidth=0.5)
    #     # prevCountData = curCountData


    # # ax1.plot(diffDates,curCountData,'k*-',linewidth=0.5)
    # # ax2.plot(diffDates,curCountData,'k*-',linewidth=0.5)



    ### Starting at last data point
    testdfRT = dfRT[dfRT.Age == '10-19']
    curDays = testdfRT.End
    prevCount = np.zeros(testdfRT.Count.shape)
    curCount = np.zeros(testdfRT.Count.shape)
    curCountData = np.zeros(dfCaseDiff['10-19'].values.shape)
    prevCountData = np.zeros(dfCaseDiff['10-19'].values.shape)

        
    # for k in range(0,len(allAges)):
    for k in range(0,len(allAges)-3):
        curAge = allAges[k]
        curColor = cmap(k)

        curCount = np.add(dfRT[dfRT.Age == curAge].Count.values,curCount)

        ax1.fill_between(curDays,curCount,prevCount,color=curColor,label=curAge)


        prevCount = curCount

        curCountData = np.add(dfCaseDiff[curAge].values,curCountData)
        ax1.plot(diffDates,curCountData,'*-',color=curColor,linewidth=0.5)
        prevCountData = curCountData
    
    nowTot = curCount[1:].sum()

    ax1.plot(curDays,curCount,'k',label='Total: '+str(int(np.round(nowTot))))
    ax1.plot(diffDates,curCountData,'k*-',linewidth=0.5,label='Data')

    ### Delayed
    testdfRTD = dfRTD[dfRTD.Age == '10-19']
    curDaysD = testdfRTD.End
    prevCountD = np.zeros(testdfRTD.Count.shape)
    curCountD = np.zeros(testdfRTD.Count.shape)
    curCountData = np.zeros(dfCaseDiff['10-19'].values.shape)
    prevCountData = np.zeros(dfCaseDiff['10-19'].values.shape)

        
    # for k in range(0,len(allAges)):
    for k in range(0,len(allAges)-3):
        curAge = allAges[k]
        curColor = cmap(k)

        curCountD = np.add(dfRTD[dfRTD.Age == curAge].Count.values,curCountD)

        ax2.fill_between(curDaysD,curCountD,prevCountD,color=curColor,label=curAge)

        prevCountD = curCountD

        # Plot data
        curCountData = np.add(dfCaseDiff[curAge].values,curCountData)
        ax2.plot(diffDates,curCountData,'*-',color=curColor,linewidth=0.5)
        prevCountData = curCountData

        # Also connect last data-point to first model-point
        ax2.plot([diffDates[-1],curDaysD.values[0]],[curCountData[-1],curCountData[-1]],':',color=curColor,linewidth=3)
        

    # Calculate the total, assuming data repeated twice
    delayTot = curCountD[1:].sum() + curCountData[-1] + curCountData[-1]

    ax2.plot(curDaysD,curCountD,'k',label='Total: '+str(int(np.round(delayTot))))
    ax2.plot(curDays,curCount,'--',label='Total,\nsom ovenfor',color='grey')
    ax2.plot(diffDates,curCountData,'k*-',linewidth=0.5,label='Data')

    ax2.plot([diffDates[-1],curDaysD.values[0]],[curCountData[-1],curCountData[-1]],'k:',linewidth=3)


    # for curAge in allAges:
        # agedfRTD = df[df.Age == curAge]
        # ax1.plot(agedf.End,agedf.Count)

    # ax1.plot(allDays,totCount,'k')
    # ax1.plot(allDaysD,totCountD)


    # Make figure nice
    curTop = ax1.get_ylim()[1]

    ax1.set_ylim(bottom=0,top=curTop)
    ax2.set_ylim(bottom=0,top=curTop)

    ax1.set_ylabel('Ugentlige nye smittetilfælde')
    ax2.set_ylabel('Ugentlige nye smittetilfælde')


    ax1.grid()
    ax2.grid()

    # ax1.legend()
    # ax2.legend(loc='upper right')
    
    ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
    ax2.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))

    ax1.set_title('Effektivt kontakttal: '+str(RT2_to_use/10).replace('.',','))

    ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.\n%B'))

    lastDate = np.datetime64('2021-09-01')
    firstDate = curDays.values[0] - np.timedelta64(21,'D')
    ax1.set_xlim([firstDate,lastDate])

    plt.tight_layout()
    if saveFigures:
        plt.savefig('figs/ForsinketFremskrivningRT'+str(RT2_to_use))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
curCount.sum()
curCountD.sum()

258545.49664983136

# Ugentlige forsinkelser

In [11]:
datafilenameD2 = 'compare_model_results.csv'
dfD2 = pd.read_csv(datafilenameD2)
dfD2.End =  pd.to_datetime(dfD2.End,format='%Y-%m-%d')
dfD2.Start =  pd.to_datetime(dfD2.Start,format='%Y-%m-%d')
dfD2['RT2'] = np.round(np.multiply(dfD2.RT,10)).astype(int)
# dfD['Hosp'] = np.multiply(dfD.Count,dfD.CaseHospRate)

In [12]:
fig,ax1 = plt.subplots()

curdf = dfD2[dfD2.RT2 == 13]


allStart = curdf.Start.unique()

cmap = cm.get_cmap('inferno',len(allStart)+1)

# for curStart in allStart:
for k in range(len(allStart)):
    curStart = allStart[k]
    thisdf = curdf[curdf.Start == curStart]
    
    curLabel = 'Fra '+pd.to_datetime(curStart).strftime('%d. %B')
    ax1.plot(thisdf.End,thisdf.Count,color=cmap(k),label = curLabel)



ax1.plot(diffDates,curCountData,'*-',linewidth=2,label='Data',color='xkcd:dark grey')


numToShow = len(allStart)

projDates = np.arange(diffDates[-1],diffDates[-1] + np.timedelta64(7*numToShow,'D'),np.timedelta64(7,'D'))

projData = curCountData[-1]*np.ones((numToShow,))
# projData = [curCountData[-1],curCountData[-1],curCountData[-1],curCountData[-1]]
ax1.plot(projDates,projData,'o:',linewidth=1,markersize=5,label='Data (forlænget)',color='xkcd:dark grey')

# ax1.plot(curdf.End,curdf.Count)
ax1.set_ylim(bottom=0)

ax1.set_ylabel('Ugentlige nye smittetilfælde')


ax1.grid()

# ax1.legend(loc='center left',bbox_to_anchor = (1.0, 0.5))
ax1.legend(loc='upper right')
# ax1.set_title('Effektivt kontakttal: '+str(RT2_to_use/10).replace('.',','))

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d.\n%B'))

lastDate = np.datetime64('2021-09-01')
firstDate = curDays.values[0] - np.timedelta64(35,'D')
ax1.set_xlim([firstDate,lastDate])


plt.tight_layout()

if saveFigures:
    plt.savefig('figs/ForsinketFremskrivningSammenlignet')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
thisdf

,Unnamed: 0,Start,End,RT,Count,CountCumul,HospCumul,RT2
679,679,2021-07-13,2021-07-13,1.3,5591.000000,0.000000,0.000000,13
688,688,2021-07-13,2021-07-20,1.3,2667.181305,2667.181305,68.467651,13
697,697,2021-07-13,2021-07-27,1.3,1636.140906,4303.322211,112.848021,13
706,706,2021-07-13,2021-08-03,1.3,849.503296,5152.825507,138.477366,13
715,715,2021-07-13,2021-08-10,1.3,344.534226,5497.359733,149.687676,13
724,724,2021-07-13,2021-08-17,1.3,96.180918,5593.540651,152.907217,13
733,733,2021-07-13,2021-08-24,1.3,15.526534,5609.067185,153.432366,13
742,742,2021-07-13,2021-08-31,1.3,1.523163,5610.590348,153.484164,13
751,751,2021-07-13,2021-09-07,1.3,0.069211,5610.659559,153.486523,13
